In [128]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models

In [129]:
#loading mnist data from tensorflow
#from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)
#mnist = tfds.load(name='mnist', split='train', as_supervised=True)
# Returns both train and test split separately
mnist_train, mnist_test = tfds.load('mnist', split=['train', 'test'], as_supervised=True)
def preprocess_image(image, label):
    image = image
    #image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0, 1] # i believe this is not in the original code
    label = tf.one_hot(label, depth=10)  # One-hot encode the label
    return image, label
mnist_train = mnist_train.map(preprocess_image)
mnist_test = mnist_test.map(preprocess_image)

In [130]:
#defining the network
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784

n_conv1 = 32
n_conv2 = 64
stride_conv1 = 1
stride_conv2 = 1
conv1_k = 5
conv2_k = 5
max_pool1_k = 2
max_pool2_k = 2

n_hidden = 1024
n_out = 10

input_size_to_hidden = (input_width//(max_pool1_k*max_pool2_k)) * (input_height//(max_pool1_k*max_pool2_k)) *n_conv2

In [131]:
#initialising the weights with random values
# NOT USED IN THIS CODE WITH KERAS IMPLEMENTATION
weights = {
    "wc1" : tf.Variable(tf.random.normal([conv1_k, conv1_k, input_channels, n_conv1])),
    "wc2" : tf.Variable(tf.random.normal([conv2_k, conv2_k, n_conv1, n_conv2])),
    "wh1" : tf.Variable(tf.random.normal([input_size_to_hidden, n_hidden])),
    "wo" : tf.Variable(tf.random.normal([n_hidden, n_out]))
}

biases = {
    "bc1" : tf.Variable(tf.random.normal([n_conv1])),
    "bc2" : tf.Variable(tf.random.normal([n_conv2])),
    "bh1" : tf.Variable(tf.random.normal([n_hidden])),
    "bo" : tf.Variable(tf.random.normal([n_out])),
}

In [132]:
#functions for layers
def conv(x, weights, bias, strides = 1):
    out = tf.nn.conv2d(x, weights, padding="SAME", strides = [1, strides, strides, 1])
    out = tf.nn.bias_add(out, bias)
    out = tf.nn.relu(out)
    return out

def maxpooling(x, k = 2):
    return tf.nn.max_pool(x, padding = "SAME", ksize = [1, k, k, 1], strides = [1, k, k, 1])

In [133]:
#function for forward prop
#This function is edited now with keras functionality. weights are now handled internally but can still be accessed with get_weights method
def cnn(x):
    # Define the layers using Keras API
    #x = layers.Reshape((input_height, input_width, input_channels))(x)
    
    conv1 = layers.Conv2D(n_conv1, kernel_size=(conv1_k, conv1_k), strides=stride_conv1, padding="same", activation="relu")(x)
    conv1_pool = layers.MaxPooling2D(pool_size=(max_pool1_k, max_pool1_k))(conv1)

    conv2 = layers.Conv2D(n_conv2, kernel_size=(conv2_k, conv2_k), strides=stride_conv2, padding="same", activation="relu")(conv1_pool)
    conv2_pool = layers.MaxPooling2D(pool_size=(max_pool2_k, max_pool2_k))(conv2)

    # Flatten the output from convolutional layers
    flatten = layers.Flatten()(conv2_pool)

    # Fully connected layer
    hidden = layers.Dense(n_hidden, activation="relu")(flatten)

    # Dropout
    dropout = layers.Dropout(0.2)(hidden)

    # Output layer (logits)
    output = layers.Dense(n_out, activation = 'softmax')(dropout)
    
    return output
    
    #x = tf.reshape(x, shape = [-1 ,input_height, input_width, input_channels])
    #conv1 = conv(x, weights['wc1'], biases['bc1'], stride_conv1)
    #conv1_pool = maxpooling(conv1, max_pool1_k)
    
    #conv2 = conv(conv1_pool, weights['wc2'], biases['bc2'], stride_conv2)
    #conv2_pool = maxpooling(conv2, max_pool2_k)
    
    #hidden_input = tf.reshape(conv2_pool, shape = [-1, input_size_to_hidden])
    #hidden_output_before_activation = tf.add(tf.matmul(hidden_input, weights['wh1']), biases['bh1'])
    #hidden_output_before_dropout = tf.nn.relu(hidden_output_before_activation)
    #hidden_output = tf.nn.dropout(hidden_output_before_dropout, keep_prob) 
   
    #output = tf.add(tf.matmul(hidden_output, weights['wo']), biases['bo']) #no softmax activation function since the loss function handles it already
    #return output

In [134]:
#x = tf.keras.Input(shape=(input_pixels,))
x = tf.keras.Input(shape=(input_height, input_width, input_channels))
y = tf.keras.Input(shape=(n_out,))
#pred is the model
pred = cnn(x)

In [135]:
model = models.Model(inputs=[x], outputs=pred)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) #changed from 0.01 to 0.001
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
train_data = mnist_train.batch(100)
test_data = mnist_test.batch(100)
# Get a single batch from the test data (a batch is a tuple of (image, label))
#single_batch = next(iter(test_data))

# single_batch is a tuple, where single_batch[0] contains the image and single_batch[1] contains the label
#image, label = single_batch

# You can print or inspect the first image and label from the batch
#print(image[0].numpy())  # The first image in the batch
#print(label[0].numpy())  # The label for the first image


In [136]:
model.fit(train_data, epochs=25)

Epoch 1/25


/opt/miniconda3/lib/python3.11/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_159']
Received: inputs=Tensor(shape=(None, 28, 28, 1))
  warnings.warn(msg)


600/600 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.8298 - loss: 4.9628
Epoch 2/25
600/600 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.9767 - loss: 0.0744
Epoch 3/25
600/600 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.9840 - loss: 0.0507
Epoch 4/25
600/600 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.9881 - loss: 0.0388
Epoch 5/25
600/600 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.9893 - loss: 0.0339
Epoch 6/25
600/600 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.9900 - loss: 0.0303
Epoch 7/25
600/600 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.9909 - loss: 0.0288
Epoch 8/25
600/600 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.9910 - loss: 0.0292
Epoch 9/25
600/600 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.9900 - loss: 0.0355
Epoch 10/25
600/600 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.9924 - loss: 0.0270
Epoch 11/25
600/600 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.9913 - loss: 0.0276
Epoch 12/25
600/600 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/ste

In [137]:
test_loss, test_acc = model.evaluate(test_data) #dropout is automatically set to 1.0 when calling model.evaluate
print(f'Test accuracy: {test_acc}')
print("summary: ")
model.summary()

100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9863 - loss: 0.1078
Test accuracy: 0.986299991607666
summary: 


Model: "functional_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_55 (InputLayer)     │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ (None, 28, 28, 32)     │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_24 (MaxPooling2D) │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_27 (Conv2D)              │ (None, 14, 14, 64)     │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_25 (MaxPooling2D) │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_12 (Flatten)            │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 1024)           │     3,212,288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 10)             │        10,250 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,823,904 (37.48 MB)

 Trainable params: 3,274,634 (12.49 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 6,549,270 (24.98 MB)